# 4. Classification (ML) Models - learning categories from data

In previous workshops, we focused on regression problems, learning how to predict continuous variables using methods like Random Forest and Neural Networks. Today, we will work on a different type of problem: **classification**. Specifically, we will use machine learning to predict a **sediment categorical characteristic**, based on its **location** and some **physical characteristics**.

Our dataset comes from the Geological Survey of the Netherlands and contains descriptions of sediments from the North Sea. Today, we will use a small, pre-processed subset of the dataset, but you can download the full dataset (and many other geological datasets!) at  [DINOloket](https://www.dinoloket.nl/en/subsurface-data). 


![DINOloket](images/5_DINOloket.png)

## 4.1 Problem Definition

In this workshop, we will use a dataset containing sample descriptions of sediments from the North Sea. When a sample is collected, the Geological Survey of the Netherlands (GDN as denoted in Dutch) follows a standard method to describe the sediment. Using this "Standard Drill Description Method" ([Standaard Boor Beschrijvingsmethode](https://www.grondwatertools.nl/sites/default/files/GDN_SBB-NITG-00-141-A-(3)_20161111.pdf)) the GDN aims to systematically capture multiple characteristics of the collected samples. This method does not only apply to marine sediments, but to any sample that is described by the GDN. Of course, some characteristics only apply to certain types of samples. 

While some of these descriptions can be made quickly, others require laboratory analysis, which is more time-consuming and resource-intensive. Today, we will try to predict one of the time-consuming measurements (i.e. **Medium sand size category**) based on **location** and some easy-to-describe **sediment properties**.

The **Medium sand size category** corresponds to **7** different categories in our dataset based on the size sand size of the sample. This measurement only applies to samples described as sand and those that have a representative portion of sand admixture. 

| Class            | Sand Median (µm)     | Code  |
|-------------------|----------------------|-------|
| Extremely fine    | 63 ≤ x < 105           | ZUF   |
| Very fine         | 105 ≤ x < 150          | ZZF   |
| Moderately fine   | 150  ≤ x < 210          | ZMF   |
| Moderately coarse | 210 ≤ x < 300          | ZMG   |
| Very coarse       | 300 ≤ x < 420          | ZZG   |
| Extremely coarse  | 420  ≤ x< 2000         | ZUG   |

**Other categories (ABM = NEN209 and ONB)**:

- Coarse category: 210 - < 2000 µm (ZGC)


Below are the predictor variables and the target variable for this exercise. Note that the sediment properties (e.g., color, calcareous portion) are also classified according to the categories in the 'Standard Drill Description Method'. If you want more details about these features, refer to the [document](https://www.grondwatertools.nl/sites/default/files/GDN_SBB-NITG-00-141-A-(3)_20161111.pdf) (information in Dutch).


| Feature Name (English)       | Feature Name (Dutch)              | Explanation                                | Reference (Page) |
|-------------------------------|------------------------------------|--------------------------------------------|------------------|
| Sample ID                    | NITG.nr                           | Sample ID                                 |                  |
| X coordinate                 | X.coordinaat                      | X coordinate (CRS:28892)                  |                  |
| Y coordinate                 | Y.coordinaat                      | Y coordinate (CRS:28892)                  |                  |
| Height with respect to NAP   | Maaiveldhoogte..m.tov.NAP         | Z coordinate (depth)                      |                  |
| Color                        | Kleur                             | Color based [SBB format L4]               | 47               |
| Calcareous portion           | Kalkgehalte                       | Calcareous content [SBB format L14]       | 75               |
| Main soil type               | Hoofdgrondsoort                   | Main soil type based [SBB format L3.1]    | 35               |
| Organic portion              | Organische Stof                   | Organic portion [SBB format: L9]          | 65               |
| Sand median class            | Zandmediaanklasse                 | Sand median [SBB format: L7.2.1]          | 52               |


### Import data and libaries

Scikit-learn already contains most of the classification tools we will need available, so we can keep using the libraries we have already seen so far. We will import the necessary ```sklearn``` tools as we need them.

In [ ]:
# Import the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

We have also gathered the data in this repository so you can directly access it. We have selected only a few variables as predictors to simplify the task.

In [ ]:
# Load the data
data_url =  "https://raw.githubusercontent.com/oriol-pomarol/codegeo_workshops/main/data/5_classification/sample_descriptions.csv"
data = pd.read_csv(data_url, delimiter=",", on_bad_lines='skip')

# Define the input and output data
input_data = data[['X.coordinate', 'Y.coordinate', 'height_NAP', 'main_soil_type']]
output_data = data['sand_median_class']

Finally, we simply need to clean the data (as it contains NaN values).

In [ ]:
# Drop the rows with missing values in the predictors
drop_rows = input_data.isnull().any(axis=1)
input_data = input_data[~drop_rows]
output_data = output_data[~drop_rows]
print('Number of samples:', len(input_data))

## 4.2 Predicting probabilities

Bridging the gap between regression and classification task might sound difficult, but it can be achieved with a few simple steps:

1. **From class to number**: To be able to use our regression models to predict classes, we need to convert those to numbers. If the classification task is binary (only two possibilities) then this is as simple as using 1 and 0. For multi-class problems a single number is not enough, so we assign each class to a vector with the same size as the number of classes. These vectors are filled with 0s except in one position correspondig to the respective class, which is filled by a 1. This is why this approach is known as *one-hot encoding*.

> **Attention**: You might be wondering why not to predict a single number and simply assign the additional classes to another value, for example to 2. This is actually a very bad idea as it would assume that your classes are ordered and would punish errors unevenly during training.

2. **From number to probability**: In the previous step we converted our class to a number, but only to 1s or 0s, but our regressor models can only predict continuous numbers. The trick here, is that instead of directly predicting the class, we predict the probability of that particular class. To convert it to our binary outputs we set a *probability threshold*, usually 0.5, for deciding between the two. For multiple classes, we can simply take the one with highest probabilty.

3. **From probability to regression**: The final step is how to make our regressor model only predict values (or vectors of values) between one and zero. For binary problems, this can be easily solved by applying the *sigmoid* function to the output of the regression model. For multi-class problems, there is another function called *softmax* that can be applied to our predicted vector to ensure that their components sum up to one, as we would expect from a set of probabilities.

Let's experiece this concept in practice by training the simplest classifier available, the logistic regression. In this case, the regressor used for predicting the probabilities is a simple multi-linear regression.

### Variable encoding

To get started, let's transform our output to a binary variable according to the coarse and fine categories. We need to assign a numerical value (1 or 0) to each of them as we have explained. Let's check how it looks at the moment.

In [ ]:
# Print the unique output values
print('Initial output unique values:', output_data.unique())

 As you can see, the word coarse is included in all the names for the coarse soil classes (duh). So, we can easily convert the output to boolean (that is, True or False) by checking if the output string contains ```'coarse'``` or not. Lastly we can check again what unique values there are in the output variable to ensure that we have performed our encoding correctly.

In [ ]:
# Transform the output to a binary number
output_binary = output_data.str.contains('coarse').astype(int)

# Print the unique output values after encoding them
print('Output unique values after encoding:', output_binary.unique())

Ah, we are not done yet unfortunately. One of the predictor variables, ```main_soil_type```, is also categorical! If we try to train with it as is we would get the following error: ```ValueError: could not convert string to float```. To prevent that, we will have to also convert it to a number. Let's see what are the unique values for this one.

In [ ]:
# Print the unique values from the main_soil_type predictor
print('Initial main_soil_type unique values:', input_data['main_soil_type'].unique())

As your can see there are four distinct categories. In this case, we don't want to convert them to binary, as we want to keep the diversity in the inputs, so we will need to use another method: **one-hot encoding**. As explained above, this means transforming the output to a vector containing all 0s except a single 1.

Because we want to use it as a predictor, not as a target, this will change a bit how we need to tackle this, as vectors are not valid inputs either. Therefore, we will need to create a separate column for each element of that vector. Another crucial difference is that we need one less column than categories in the predictor variable to avoid multicollinearity (highly correlated input variables).

In practice, we already have a function in ```pandas``` which will apply one-hot encoding to the variables of our choice. Let's apply it to ```main_soil_type``` and see how the outcome looks like.

In [ ]:
# Use pandas get_dummies to encode the main_soil_type predictor
input_encoded = pd.get_dummies(input_data, columns=['main_soil_type'], drop_first=True)
input_encoded

### Classifier training and prediction

Perfect, now that we have our inputs and outputs as numbers, we can train the logistic regression model on our binary data. As you know, first we will need to split between train and test data to have an independent validation set. Before training, we are going to **normalize** the data. This step is necessary to be able compare the coefficients of the logistic regression and to facilitate training, and can be easily done using the ```StandardScaler()``` class from Scikit-learn. Once the training is complete, we can check the linear coefficients of such model.

In [ ]:
# Split the data into training and test sets
from sklearn.model_selection import train_test_split
X_train_binary, X_test_binary, y_train_binary, y_test_binary \
    = train_test_split(input_encoded, output_binary, test_size=0.2,
                       shuffle=True, random_state=10)

# Normalize the data using the standard scaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_binary_scaled = scaler.fit_transform(X_train_binary)

# Define the logistic regression model and train it with our data
from sklearn.linear_model import LogisticRegression
log_reg_binary = LogisticRegression()
log_reg_binary.fit(X_train_binary_scaled, y_train_binary)

# Print the coefficients of the linear regression model
coefficients = log_reg_binary.coef_[0]
feature_names = X_train_binary.columns
pd.DataFrame({'Coefficient': coefficients}, index=feature_names)

This already gives us very important information as to which variables are positively correlated with course sand grains (our output) and which negatively, according to the sign of their coefficients. Because we normalized the data, we also get an idea of which predictors are more impactful in the decision. For example, the heigth seems to be a very good indicator of coarse sand grains.

Now let's try to calculate what the prediction should be for the first point in the test set by following the steps that we introduced before - but now the other way around!

In [ ]:
# Transform the test data using the same scaler
X_test_binary_scaled = scaler.transform(X_test_binary)

# Take the first sample from the test data
X = X_test_binary_scaled[0:1]

# Calculate the output of the regression model using the coefficients
regression_output = (np.dot(X, coefficients) + log_reg_binary.intercept_)[0]

# Calculate the probability by using the sigmoid function
probability = 1 / (1 + np.exp(-regression_output))

# Check if the probability exceeds the 0.5 treshold
probability_threshold = 0.5
predicted_class = int(probability > probability_threshold)

print(f'Regression: {regression_output:.4f} -> Probability: {probability:.4f}' 
      f' -> Predicted class: {predicted_class}')

We can also access the probabilities when we make predictions using the regression model by using the ```.predict_proba()``` method. If we use the usual ```.predict()``` method, a probability threshold of 0.5 is used by default. Let's check that those values correspond to the ones that we have just calculated.

In [ ]:
# Check that the logistic regression returns the same prediction
log_reg_proba = log_reg_binary.predict_proba(X)
log_reg_class = log_reg_binary.predict(X)

# Print the results as a dataframe with both calculated and logistic regression results
pd.DataFrame({'Class': [predicted_class, log_reg_class[0]], 
              'Probability': [probability, log_reg_proba[0, 1]]}, 
             index=['Calculated', 'Logistic regression'])

Great! Now that we understood how the prediction was made, let's make a prediction for all the available data to see how well we are doing. To make it more interesting we will check the predictions for a probability threshold of 0.25, 0.5 and 0.75. We can enforce them by predicting the probabilities, then comparing them to the thresholds. How do you think this will influence the results?

In [ ]:
# Plot a map of the data classes for the original data
plt.figure(figsize=(5, 5))
scatter = plt.scatter(input_encoded['X.coordinate'],
                      input_encoded['Y.coordinate'],
                      c=output_binary, cmap='coolwarm')

# Add labels and title to the plot
plt.xlabel('X coordinate')
plt.ylabel('Y coordinate')
plt.title('Original data classes')

# Add a legend and show the plot
legend = scatter.legend_elements()[0]
plt.legend(legend, ['Fine', 'Coarse'], loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

In [ ]:
# Scale the original data
input_encoded_scaled = scaler.transform(input_encoded)

# Make predictions using a probability threshold of 0.25, 0.5 and 0.75
probability_thresholds = [0.25, 0.5, 0.75]
predicted_classes = []
for threshold in probability_thresholds:
    predicted_probability = log_reg_binary.predict_proba(input_encoded_scaled)[:, 1]
    predicted_classes.append((predicted_probability > threshold).astype(int))

# Plot a map of the predicted classes with different probability thresholds
fig, axes = plt.subplots(1, 3, figsize=(15, 5), sharex=True, sharey=True)

for i, (ax, threshold) in enumerate(zip(axes, probability_thresholds)):
    scatter = ax.scatter(input_encoded['X.coordinate'], input_encoded['Y.coordinate'], 
                         c=predicted_classes[i], cmap='coolwarm')
    ax.set_xlabel('X coordinate')
    ax.set_ylabel('Y coordinate')
    ax.set_title(f'Predicted classes with threshold {threshold}')

plt.show()

As you might have expected, when we increase the probability threshold we predict the class we assigned to 0 (fine grained sands) more often. Essentially, with higher probability threshold we require the model to be increasingly confident that the data is coarse grained sand to classify it as such. We might want to do that, for example, if it would be important that we don't missclassify any fine grained sands. More on that in the following sections. In general, it is best to tune the probability threshold to ensure that we don't overpredict one of the variables, especially if there are many more instances of one class in the data. Keep tuned for a future workshop on how to deal with imbalanced data if you are interested in the topic!

## 4.3 Evaluating the models

### The confusion matrix

The best way to visualize the results of a classifier model is through a "confusion matrix". This is nothing more than a table which columns indicate the classes predicted by the ML model and which rows are the actual classes from the data. This is what it looks like for a binary problem:

| Class           | Predicted Positive         | Predicted Negative         |
|------------------------|----------------------------|----------------------------|
| **Actual Positive**        | **TP** (True Positive)     | **FN** (False Negative)    |
| **Actual Negative**        | **FP** (False Positive)    | **TN** (True Negative)     |

Although the two binary classes are usually referred to as "positive" and "negative", it can be any two type of classes, so negative does not necessarily imply *bad*. In our case, for example, the classes refer to either coarse or fine grained sands, so the "positive" class is simply the one we assign to 1. We could think of it as if we are predicing whether there are coarse grained sands (positive) or not (negative). Let's generate a confusion matrix with the predictions from our logistic model we trained before **on the test data**. This can be easily computed with the ```confusion_matrix``` function in ```sklearn.metrics```.

In [ ]:
# Make predictions with the logistic regression model for the test set
y_pred_binary = log_reg_binary.predict(X_test_binary_scaled)

# Create the confusion matrix with the logistic regression model predictions
from sklearn.metrics import ConfusionMatrixDisplay
ConfusionMatrixDisplay.from_predictions(y_test_binary, y_pred_binary)
plt.show()

While simply displaying the confusion matrix is already quite informative of model performance, sometimes we want to test on specific metrics. Many of the common metrics used to evaluate the performance of classifier models can actually be computed from the confusion matrix directly. Here are some examples:

* **Accuracy**: Probably the best known classification metric, it evaluates the percentage of samples that were classified into the correct class.

$$Accuracy = \frac{TP+TN}{TP+FP+TN+FN}$$

* **Precision**: A high precision indicates there were not many false alarms of a specific (in binary, the positive) class.

$$Precision = \frac{TP}{TP+FP}$$

* **Recall**: A high recall indicates that there were not many missed cases of a specific (in binary, the positive) class. 

$$Recall = \frac{TP}{TP+FN}$$

First, try to compute those metrics with pen and paper from the confusion matrix that we just saw. When you are done, you can compare your results with those obtained by using the corresponding Scikit-learn functions.

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

# Compute the accuracy of the logistic regression model
accuracy = accuracy_score(y_test_binary, y_pred_binary)
print(f"The accuracy of the logistic regression model is: {accuracy}")

# Compute the precision of the logistic regression model
precision = precision_score(y_test_binary, y_pred_binary)
print(f"The precision of the logistic regression model is: {precision}")

# Compute the recall of the logistic regression model
recall = recall_score(y_test_binary, y_pred_binary)
print(f"The recall of the logistic regression model is: {recall}")


### A more balanced metric

While accuracy is the most straightforward way to determine the performance of a classification model, it might not always be the most suitable. This is especially true when the data that is being predicted is imbalanced, that is, when we have many more instances of one class than the rest. Then, it is common that the classifier learns to predict in favour of the majority class, performing really poorly in the rest. For those cases, there is a better metric that we can use that combines both precision and recall to obtain a more balanced metric for performance, the **F1-score**.

$$F1\ score = 2 \cdot \frac{Precision \cdot Recall}{Precision + Recall}$$

Let's compare accuracy and F1-score for the different probability thresholds that we tested in the previous chapter. Which of the two do you feel better relates to model performance?

In [ ]:
from sklearn.metrics import f1_score

# Calculate the F1 score for the results with different probability thresholds
for i, threshold in enumerate(probability_thresholds):

    # Get the test indexes of the test data from the input_encoded dataframe
    test_indixes = input_encoded.index.get_indexer(y_test_binary.index)
    predicted_test_binary = predicted_classes[i][test_indixes]

    # Calculate the F1 score and accuracy for the given threshold
    f1 = f1_score(y_test_binary, predicted_test_binary)
    accuracy = accuracy_score(y_test_binary, predicted_test_binary)
    print(f"Threshold: {threshold}\nAccuracy = {accuracy:.2f}; F1 score = {f1:.2f}\n")

## 4.4 Multi-class tasks

### Predicting multiple classes

While binary problems are relatively common, many times we would like to predict multiple classes. This is the case, for example, if we want to determine the specific sand grain size category from our data. To make things interesting, we are also going to use more powerful ML algorithms; we will compare the results of:

* K-Nearest Neighbors (KNN): This is one of the most commonly known examples of **lazy learners**, which spend little time training but take longer for inference as they have to access the training data. This particular algorithm checks the K nearest neighbours (5 by default) and makes a prediction based on those.

* Random Forest (RF): We have already discussed this very common ML algorithm for regressin tasks. Due to the decision-based nature of this model, it works very nicely out of the box with categorical data, making it especially suitable for classification tasks.

Again we will need to generate a train and test data with the new outputs and train the model.

In [ ]:
# Train the ANN and RF models
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

# Split the data into training and test sets
X_train_multi, X_test_multi, y_train_multi, y_test_multi = \
    train_test_split(input_encoded, output_data, test_size=0.2,
                     shuffle=True, random_state=10)

# Train the random forest model
rf = RandomForestClassifier(random_state=10)
rf.fit(X_train_multi, y_train_multi)

# Train the k-neighbors classifier model
kn = KNeighborsClassifier()
kn.fit(X_train_multi, y_train_multi)

Now that the models are trained, we can show the predictions on a map.

In [ ]:
# Make predictions for the whole dataset
y_pred_rf = rf.predict(input_encoded)
y_pred_kn = kn.predict(input_encoded)

# Create subplots with shared x and y axes
fig, axes = plt.subplots(1, 3, figsize=(15, 5), sharex=True, sharey=True)

# Plot the original data classes
scatter = axes[0].scatter(input_encoded['X.coordinate'], input_encoded['Y.coordinate'],
                          c=pd.Categorical(output_data).codes, cmap='tab20')
axes[0].set_xlabel('X coordinate')
axes[0].set_ylabel('Y coordinate')
axes[0].set_title('Original data classes')

# Plot the predicted classes for RF and KNN
for i, (ax, y_pred) in enumerate(zip(axes[1:], [y_pred_rf, y_pred_kn])):
    ax.scatter(input_encoded['X.coordinate'], input_encoded['Y.coordinate'],
                         c=pd.Categorical(output_data).codes, cmap='tab20')
    ax.set_xlabel('X coordinate')
    ax.set_ylabel('Y coordinate')
    ax.set_title(f'Predicted classes with {"RF" if i == 0 else "KNN"}')

# Add a legend and show the plot
legend = scatter.legend_elements()[0]
output_classes = pd.Categorical(output_data).categories
fig.legend(legend, output_classes, loc='center left', bbox_to_anchor=(0.9, 0.5))
plt.show()

### Generalizing the metrics

The map looks rather similar at first glance. To further evaluate the models, we can use the same tools that we did for the binary problem with some slight differences, for example the confusion matrix. Let's see how it looks for multiple classes. 

In [ ]:
# Predict the classes for the test set
y_pred_rf_test = rf.predict(X_test_multi)
y_pred_kn_test = kn.predict(X_test_multi)

# Plot the confusion matrices
fig, axes = plt.subplots(1, 2, figsize=(10, 5), sharex=True, sharey=True)

# Plot the confusion matrix for the Random Forest
ConfusionMatrixDisplay.from_predictions(y_test_multi, y_pred_rf_test, ax=axes[0])
axes[0].set_title('Random Forest')
axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=45, ha='right')

# Plot the confusion matrix for the K-Neighbors Classifier
ConfusionMatrixDisplay.from_predictions(y_test_multi, y_pred_kn_test, ax=axes[1])
axes[1].set_title('K-Neighbors Classifier')
axes[1].set_xticklabels(axes[1].get_xticklabels(), rotation=45, ha='right')

plt.show()

Same as with the binary case, the correctly classified data points are stored in the diagonal. However, as we keep adding classes the confusion matrix gets more and more cluttered, increasing the usefulness of using metrics instead. All the metrix that we have seen previously can be generalized to multi-class problems, with some slight differences. For the accuracy, for example, we will simply need to sum all elements in the diagonal and divide by the total data sample size.

In [ ]:
# Compute the accuracy for each of the models
accuracy_rf = accuracy_score(y_test_multi, y_pred_rf_test)
accuracy_kn = accuracy_score(y_test_multi, y_pred_kn_test)

print(f"Random Forest accuracy: {accuracy_rf:.2f}")
print(f"K-Neighbors Classifier accuracy: {accuracy_kn:.2f}")

Recall and precision (and therefore F1-score) are now defined for each class, and need to be averaged for getting a general metric for a model output. However, sometimes we might want to know those metrics only for specific classes. In our example, we might be especially concerned about correctly classifying extremely fine-grained sands, since those can easily infiltrate in machinery that might be installed on these sites reducing their useful lifespan greatly. Which of the two variables should we then compare?

In [ ]:
# Compute the recall of both models for the extremely fine grain size
recall_rf = recall_score(y_test_multi, y_pred_rf_test, labels=['extremely fine'], average=None)
recall_kn = recall_score(y_test_multi, y_pred_kn_test, labels=['extremely fine'], average=None)

print(f"Random Forest recall for extremely fine grain size: {recall_rf:.2f}")
print(f"K-Neighbors Classifier recall for extremely fine grain size: {recall_kn:.2f}")

## 4.5 Final remarks

In this workshop you have learnt the basics of how to tackle a classification task, from the output definition to training and of course evaluating your model. To cement this knowledge, try and use the same data but choose another variable as your target, for example the main soil type. You can re-use some of the code above. If you feel like having a bit more of a challenge, include also the color column as an additional predictor. Be careful as it is also categorical (and with many categories indeed), so you will need to encode it first. Good luck!

In [ ]:
# Define the predictor and target variables

# Split between train and test sets

# Train the classification model of your choice

# Evaluate the model performance on the test set
